In [1]:
import torch
import torch.nn as nn

In [6]:
class ResidualBlock(nn.Module):#残差ブロック
    def __init__(self,in_channels,out_channels):
        super().__init__()
        #今回は複雑になるので使用する層を個別で書いてforward()にて実行する
        self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1)
        self.conv2=nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1)
        self.conv3=nn.Conv2d(in_channels,out_channels,kernel_size=1,padding=0)#チャネル数だけを変化させる
        self.bn=nn.BatchNorm2d(out_channels)#正規化をすることで過学習を防ぐ
        self.relu=nn.ReLU()
    def shortcut(self,x):
        x=self.conv3(x)#チャネル数を合わせる
        x=self.bn(x)
        return x
    def forward(self,x):
        identity=x
        x=self.conv1(x)
        x=self.bn(x)
        x=self.relu(x)
        x=self.conv2(x)
        x=self.bn(x)
        x+=self.shortcut(identity)
        return x


In [15]:
class ResNet(nn.Module):
    def __init__(self,block):
        super().__init__()
        self.linear=nn.Linear(in_features=28*28*64,out_features=10)
        self.layer=self._make_layer(block,3,3,64)
    def _make_layer(self,block,num_residual_blocks,in_channels,out_channels):
        layers=[]
        for i in range(num_residual_blocks):
            if i==0:
                layers.append(block(in_channels,out_channels))
            else:
                layers.append(block(out_channels,out_channels))
        return nn.Sequential(*layers)#*は展開して渡し新たなSequentialオブジェクトを作成
    def forward(self,x):
        x=self.layer(x)
        x=x.view(x.size(0),-1)#linear層に代入ができないから一次元のベクトルに変更
        x=self.linear(x)
        return x

In [16]:
model=ResNet(ResidualBlock)

In [17]:
model

ResNet(
  (linear): Linear(in_features=50176, out_features=10, bias=True)
  (layer): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (2): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1,

In [18]:
x_test=torch.randn(32,3,28,28)#バッチサイズ,チャネル数,画像のサイズ,画像のサイズ

In [19]:
x_test

tensor([[[[ 0.5943,  1.1204,  1.8095,  ..., -0.5899, -0.3983,  1.0795],
          [ 0.0177,  0.8933,  0.2236,  ...,  0.8265, -0.1921, -1.6521],
          [ 0.3773, -0.4772, -2.5618,  ...,  0.0638, -0.2262, -0.3469],
          ...,
          [-0.5048, -1.3692, -1.4807,  ..., -2.0944,  0.8767, -0.5872],
          [ 1.0270,  0.2775, -0.1784,  ...,  1.7257,  0.2197, -0.0243],
          [ 1.5600, -0.1153, -1.1236,  ...,  0.1127, -0.0553, -0.0250]],

         [[-0.2645,  1.6598, -0.3210,  ...,  0.9597, -2.1224, -0.4384],
          [ 0.2968,  0.2369, -1.7018,  ..., -0.3090,  0.4143,  0.0071],
          [-0.0725, -0.3885, -0.1598,  ...,  0.2511,  0.4688,  1.2905],
          ...,
          [-0.9256,  0.8655,  0.3090,  ..., -0.0142, -1.7856, -1.2953],
          [ 0.4716, -0.2851,  2.1642,  ...,  1.1736, -0.1196,  0.2952],
          [-1.4098,  2.0071,  0.4120,  ...,  0.2645,  0.7265,  1.5767]],

         [[-0.0424, -1.1871, -1.3104,  ...,  0.9320,  0.8474, -0.9172],
          [ 0.5250,  0.0543, -

In [20]:
output=model(x_test)

In [21]:
output.size()

torch.Size([32, 10])